About the dataset:

1. id: unique id for a news article
2. Title: the title of a news article
3. Author: author of the news article
4. Text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:

1: Fake news

0: real news

nltk --->Natural language tool kit
stemming takes the word and removes the head and tail so it makes world easier
TfidfVectorizer--> convert the functions into feature vector...

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

#preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline,Pipeline

#Models
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

#Cross validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

#Hyper parameter tuning
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

#Accuracy score
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix,classification_report

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Manoj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('english'))
#These are the stop words

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
#Data  Preprocessing

news_df = pd.read_csv(r'G:\Data analyst by Shashank\Project DATA_SETS\news_train.csv')
news_df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [5]:
news_df.shape

(20800, 5)

In [6]:
news_df.isnull().sum()
#Checking the missing values

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
#Replacing the null values with impty strings

news_df = news_df.fillna('')

In [8]:
#Lets merge the title and author column as content

news_df['content'] = news_df['author'] +' ' + news_df['title']

In [9]:
news_df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [10]:
#We can drop the non required columns or we can select a specific column which we 
# wanted to build model

x= news_df.drop(['label','id'], axis=1)
y = news_df['label']

In [11]:
print(x.shape)
print(y.shape)

(20800, 4)
(20800,)


#Lets do the stemming
stemming is the process of reducing a word to it's root word

eg: actor, actress, acting 
we get after stemming -->act

In [12]:
port_stem = PorterStemmer()

In [13]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [14]:
news_df['content'] = news_df['content'].apply(stemming)

In [15]:
#Seperating the data and label since we want only the content not values 
x = news_df['content']
y = news_df['label']

In [16]:
print(x.shape)
print(y.shape)

(20800,)
(20800,)


In [17]:
#Since computer cannot understand the text we convert content text into number
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(x)


In [18]:
print(X.shape)

(20800, 17128)


In [19]:
#Splitting the data into training and test data
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=3)
print(x_train.shape)
print(y_train.shape)

(16640, 17128)
(16640,)


In [20]:
model1 = LogisticRegression()
pipe = make_pipeline(StandardScaler(with_mean=False),LogisticRegression())
pipe.fit(x_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('logisticregression', LogisticRegression())])

In [21]:
#Evaluation

#Accuracy score on training data
x_train_predict = pipe.predict(x_train)
training_data_accuracy = accuracy_score(x_train_predict,y_train)
print(f'TrainingData Accuracy : {training_data_accuracy}')

TrainingData Accuracy : 1.0


In [22]:
#testing score on test data
x_test_predict = pipe.predict(x_test)
test_data_accuracy = accuracy_score(x_test_predict,y_test)
print(f'Test Data Accuracy : {test_data_accuracy}')
print(f'Confusion matrix:{confusion_matrix(x_test_predict,y_test)}')
print(f'Classification report :{classification_report(x_test_predict,y_test)}')

Test Data Accuracy : 0.9860576923076924
Confusion matrix:[[2051   32]
 [  26 2051]]
Classification report :              precision    recall  f1-score   support

           0       0.99      0.98      0.99      2083
           1       0.98      0.99      0.99      2077

    accuracy                           0.99      4160
   macro avg       0.99      0.99      0.99      4160
weighted avg       0.99      0.99      0.99      4160



In [28]:
model = []
model.append(LogisticRegression())
model.append(DecisionTreeClassifier())
model.append(SVC())
model.append(RandomForestClassifier())
model.append(KNeighborsClassifier())

In [29]:
def testing_model(model):
    for x in range(len(model)):
        model[x].fit(x_train,y_train)
        x_train_predict = model[x].predict(x_train)
        training_accuracy = accuracy_score(x_train_predict,y_train)
        print(f'Training accuracy of {model[x]} is:{training_accuracy}')
        x_test_predict = model[x].predict(x_test)
        test_accuracy = accuracy_score(x_test_predict, y_test)
        print(f'Testing accuracy of {model[x]} is:{test_accuracy}')
        print('*'*30)
    
    
        

In [30]:
testing_model(model)

Training accuracy of LogisticRegression() is:0.9861778846153846
Testing accuracy of LogisticRegression() is:0.9764423076923077
******************************
Training accuracy of DecisionTreeClassifier() is:1.0
Testing accuracy of DecisionTreeClassifier() is:0.9939903846153846
******************************
Training accuracy of SVC() is:0.9990384615384615
Testing accuracy of SVC() is:0.9918269230769231
******************************
Training accuracy of RandomForestClassifier() is:1.0
Testing accuracy of RandomForestClassifier() is:0.9930288461538461
******************************
Training accuracy of KNeighborsClassifier() is:0.5357572115384616
Testing accuracy of KNeighborsClassifier() is:0.5271634615384615
******************************


In [31]:
#Doing cross_val_score
#I am using stratified k fold since we are using Classification problem

def cross_val_scoring(model):
    for x in range(len(model)):
        skf=StratifiedKFold(n_splits=3)
        cross_val =cross_val_score(estimator=model[x],X=X, y=y,cv=skf)
        print(f' {model[x]} cross_val_score accuracy is :{np.mean(cross_val)}')
        print('*'*30)

In [32]:
cross_val_scoring(model)

 LogisticRegression() cross_val_score accuracy is :0.9742307783567671
******************************
 DecisionTreeClassifier() cross_val_score accuracy is :0.991057720001709
******************************
 SVC() cross_val_score accuracy is :0.9871635169498302
******************************
 RandomForestClassifier() cross_val_score accuracy is :0.992067376994021
******************************
 KNeighborsClassifier() cross_val_score accuracy is :0.5204325514140987
******************************


******************Yaaaaaa*******************
Random Forest works best but it's slow :)